In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [3]:
input_size = 28
num_classes = 10
num_epochs = 3
batch_size = 64

train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)

test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

# normalize=transforms.Normalize(mean=[.5,.5,.5],std=[.5,.5,.5])  #规范化
# transform=transforms.Compose([                                  #数据处理
#     transforms.Resize((64,64)),
#     transforms.ToTensor(),
#     normalize
# ])

# train_dataset = datasets.ImageFolder(r'./data/train_sample', transform=transform)

# test_dataset = datasets.ImageFolder(r'./data/test_sample', transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels = 1,
                out_channels = 16,
                kernel_size = 5,
                stride = 1,
                padding = 2,
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        self.out = nn.Linear(32 * 7 * 7, 10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        output = self.out(x)
        
        return output
        


In [5]:
def accuracy(predictions, labels):
    pred = torch.max(predictions.data, 1)[1]
    rights = pred.eq(labels.data.view_as(pred)).sum()
    return rights, len(labels)

In [6]:


net = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

for epoch in range(num_epochs):
    train_rights = []
    
    for batch_idx, (data, target) in enumerate(train_loader):
        net.train()
        output = net(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        right = accuracy(output, target)
        train_rights.append(right)
        
        if batch_idx % 100 == 0:
            net.eval()
            val_rights = []
            
            for (data, target) in test_loader:
                output = net(data)
                right = accuracy(output, target)
                val_rights.append(right)
                
            train_r = (sum(tup[0] for tup in train_rights), sum(tup[1] for tup in train_rights))
            val_r = (sum(tup[0] for tup in val_rights), sum(tup[1] for tup in val_rights))
            
            print('當前epoch: {} [{}/{} ({:.0f}%)]\t損失: {:.6f}\t訓練習準確率: {:.2f}%\t測試習準確率: {:.2f}'.format(
                 epoch, batch_idx * batch_size, len(train_loader.dataset), 100. * batch_idx / len(train_loader), loss.data, 100. * train_r[0].numpy() / train_r[1], 100. * val_r[0].numpy() / val_r[1]))

當前epoch: 0 [0/60000 (0%)]	損失: 2.316225	訓練習準確率: 3.12%	測試習準確率: 11.44
當前epoch: 0 [6400/60000 (11%)]	損失: 0.161228	訓練習準確率: 73.44%	測試習準確率: 91.78
當前epoch: 0 [12800/60000 (21%)]	損失: 0.422414	訓練習準確率: 82.77%	測試習準確率: 93.77
當前epoch: 0 [19200/60000 (32%)]	損失: 0.096513	訓練習準確率: 86.72%	測試習準確率: 95.65
當前epoch: 0 [25600/60000 (43%)]	損失: 0.248052	訓練習準確率: 89.09%	測試習準確率: 96.84
當前epoch: 0 [32000/60000 (53%)]	損失: 0.259145	訓練習準確率: 90.54%	測試習準確率: 97.49
當前epoch: 0 [38400/60000 (64%)]	損失: 0.128766	訓練習準確率: 91.56%	測試習準確率: 97.44
當前epoch: 0 [44800/60000 (75%)]	損失: 0.043851	訓練習準確率: 92.39%	測試習準確率: 97.79
當前epoch: 0 [51200/60000 (85%)]	損失: 0.020410	訓練習準確率: 93.00%	測試習準確率: 97.86
當前epoch: 0 [57600/60000 (96%)]	損失: 0.072850	訓練習準確率: 93.52%	測試習準確率: 98.15
當前epoch: 1 [0/60000 (0%)]	損失: 0.074076	訓練習準確率: 98.44%	測試習準確率: 98.20
當前epoch: 1 [6400/60000 (11%)]	損失: 0.028416	訓練習準確率: 97.85%	測試習準確率: 97.75
當前epoch: 1 [12800/60000 (21%)]	損失: 0.103409	訓練習準確率: 97.90%	測試習準確率: 98.30
當前epoch: 1 [19200/60000 (32%)]	損失: 0.036989	訓練習準確率: 97.98%	測試習準確